<a href="https://colab.research.google.com/github/mathengem/Algorithmic-Trading-Backtesting-in-python/blob/main/z3hash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 9.1 MB/s eta 0:00:00


In [ ]:
import hashlib
from z3 import *

# Function to perform SHA-256 hashing
def sha256(data):
    return hashlib.sha256(data).digest()

# Function to convert bytes to BitVec
def bytes_to_bitvec(data):
    return BitVecVal(int.from_bytes(data, byteorder='big'), len(data) * 8)

# Known part of the block header (first 608 bits)
block_header_no_nonce = b'\x00' * 76  # Example: 608 bits (76 bytes) of known data

# Target condition: the first 64 bits of the hash must be zero
target_condition = 0

# Iterate over possible nonce values
for nonce_value in range(2**32):
    # Create the full block header by appending the nonce
    block_header = block_header_no_nonce + nonce_value.to_bytes(4, byteorder='big')

    # Compute the double SHA-256 hash
    first_hash = sha256(block_header)
    second_hash = sha256(first_hash)

    # Convert the first 64 bits of the hash to an integer
    first_64_bits = int.from_bytes(second_hash[:8], byteorder='big')

    # Check if the hash meets the target condition
    if first_64_bits == target_condition:
        print(f"Solution found: Nonce = {nonce_value}")
        print(f"Hash = {second_hash.hex()}")
        break
else:
    print("No solution found.")

# Use Z3 to confirm constraints if needed
solver = Solver()

# Create a BitVec for the nonce
nonce = BitVec('nonce', 32)

# Create a fixed BitVecVal for the known beginning of the block
prefix = bytes_to_bitvec(block_header_no_nonce)

# Hash input is formed by concatenating the prefix with the nonce
input_message = Concat(prefix, nonce)

# Define a symbolic SHA-256 function
def symbolic_sha256(bitvec):
    # Convert BitVec to bytes
    bitvec_bytes = BitVecVal(int(bitvec.as_long()), bitvec.size())
    # Perform SHA-256 hashing
    hash_bytes = sha256(bitvec_bytes.as_long().to_bytes(bitvec.size() // 8, byteorder='big'))
    # Convert hash bytes back to BitVec
    return BitVecVal(int.from_bytes(hash_bytes, byteorder='big'), 256)

# Run the input through SHA-256 to get a symbolic BitVec hash digest
digest = symbolic_sha256(input_message)

# Run through SHA-256 a second time
second_digest = symbolic_sha256(digest)

# Create a variable for the hash output, which is unknown
hash_guess = BitVec('hash_guess', 256)

# Add solver constraint that this variable must equal the digest
solver.add(hash_guess == second_digest)

# Add a constraint to the solver that the beginning of the hash must have some number of zeros, e.g., 64
hash_beginning = Extract(63, 0, hash_guess)
solver.add(hash_beginning == 0)

# Check if the problem has a solution, and print it
if solver.check() == sat:
    model = solver.model()
    print("Solution found:")
    print(model)
    nonce_value = model[nonce].as_long()
    print(f"Nonce: {nonce_value}")
else:
    print("No solution found.")

In [ ]:
import hashlib
from z3 import *
from IPython.display import clear_output

# Function to perform SHA-256 hashing
def sha256(data):
    return hashlib.sha256(data).digest()

# Function to convert bytes to BitVec
def bytes_to_bitvec(data):
    return BitVecVal(int.from_bytes(data, byteorder='big'), len(data) * 8)

# Known part of the block header (first 608 bits)
block_header_no_nonce = b'\x00' * 76  # Example: 608 bits (76 bytes) of known data

# Target condition: the first 64 bits of the hash must be zero
target_condition = 0

# Iterate over possible nonce values
for nonce_value in range(2**32):
    # Create the full block header by appending the nonce
    block_header = block_header_no_nonce + nonce_value.to_bytes(4, byteorder='big')

    # Compute the double SHA-256 hash
    first_hash = sha256(block_header)
    second_hash = sha256(first_hash)

    # Convert the first 64 bits of the hash to an integer
    first_64_bits = int.from_bytes(second_hash[:8], byteorder='big')

    # Clear the previous output
    clear_output(wait=True)

    # Print the current nonce and the first 64 bits of the hash
    print(f"Nonce: {nonce_value}")
    print(f"First 64 bits of hash: {first_64_bits:016x}")

    # Check if the hash meets the target condition
    if first_64_bits == target_condition:
        print(f"Solution found: Nonce = {nonce_value}")
        print(f"Hash = {second_hash.hex()}")
        break
else:
    print("No solution found.")

# Use Z3 to confirm constraints if needed
solver = Solver()

# Create a BitVec for the nonce
nonce = BitVec('nonce', 32)

# Create a fixed BitVecVal for the known beginning of the block
prefix = bytes_to_bitvec(block_header_no_nonce)

# Hash input is formed by concatenating the prefix with the nonce
input_message = Concat(prefix, nonce)

# Define a symbolic SHA-256 function
def symbolic_sha256(bitvec):
    # Convert BitVec to bytes
    bitvec_bytes = BitVecVal(int(bitvec.as_long()), bitvec.size())
    # Perform SHA-256 hashing
    hash_bytes = sha256(bitvec_bytes.as_long().to_bytes(bitvec.size() // 8, byteorder='big'))
    # Convert hash bytes back to BitVec
    return BitVecVal(int.from_bytes(hash_bytes, byteorder='big'), 256)

# Run the input through SHA-256 to get a symbolic BitVec hash digest
digest = symbolic_sha256(input_message)

# Run through SHA-256 a second time
second_digest = symbolic_sha256(digest)

# Create a variable for the hash output, which is unknown
hash_guess = BitVec('hash_guess', 256)

# Add solver constraint that this variable must equal the digest
solver.add(hash_guess == second_digest)

# Add a constraint to the solver that the beginning of the hash must have some number of zeros, e.g., 64
hash_beginning = Extract(63, 0, hash_guess)
solver.add(hash_beginning == 0)

# Check if the problem has a solution, and print it
if solver.check() == sat:
    model = solver.model()
    print("Solution found:")
    print(model)
    nonce_value = model[nonce].as_long()
    print(f"Nonce: {nonce_value}")
else:
    print("No solution found.")

Nonce: 187936
First 64 bits of hash: 9f72a54bbb0e2d40


In [1]:
import hashlib
from z3 import *
from IPython.display import clear_output

# Function to perform SHA-256 hashing
def sha256(data):
    return hashlib.sha256(data).digest()

# Function to convert bytes to BitVec
def bytes_to_bitvec(data):
    return BitVecVal(int.from_bytes(data, byteorder='big'), len(data) * 8)

# Known part of the block header (first 608 bits)
block_header_no_nonce = b'\x00' * 76  # Example: 608 bits (76 bytes) of known data

# Target condition: the first 64 bits of the hash must be zero
target_condition = 0

# File to save the start and stop nonce values
start_stop_file = 'start_stop.txt'

# Function to read the last saved nonce value
def read_last_nonce():
    try:
        with open(start_stop_file, 'r') as file:
            return int(file.read().strip())
    except FileNotFoundError:
        return 0

# Function to save the current nonce value
def save_current_nonce(nonce_value):
    with open(start_stop_file, 'w') as file:
        file.write(str(nonce_value))

# Read the last saved nonce value
start_nonce = read_last_nonce()

# Iterate over possible nonce values starting from the last saved nonce
for nonce_value in range(start_nonce, 2**32):
    # Create the full block header by appending the nonce
    block_header = block_header_no_nonce + nonce_value.to_bytes(4, byteorder='big')

    # Compute the double SHA-256 hash
    first_hash = sha256(block_header)
    second_hash = sha256(first_hash)

    # Convert the first 64 bits of the hash to an integer
    first_64_bits = int.from_bytes(second_hash[:8], byteorder='big')

    # Clear the previous output
    clear_output(wait=True)

    # Print the current nonce and the first 64 bits of the hash
    print(f"Nonce: {nonce_value}")
    print(f"First 64 bits of hash: {first_64_bits:016x}")

    # Save the current nonce value
    save_current_nonce(nonce_value)

    # Check if the hash meets the target condition
    if first_64_bits == target_condition:
        print(f"Solution found: Nonce = {nonce_value}")
        print(f"Hash = {second_hash.hex()}")
        break
else:
    print("No solution found.")

# Use Z3 to confirm constraints if needed
solver = Solver()

# Create a BitVec for the nonce
nonce = BitVec('nonce', 32)

# Create a fixed BitVecVal for the known beginning of the block
prefix = bytes_to_bitvec(block_header_no_nonce)

# Hash input is formed by concatenating the prefix with the nonce
input_message = Concat(prefix, nonce)

# Define a symbolic SHA-256 function
def symbolic_sha256(bitvec):
    # Convert BitVec to bytes
    bitvec_bytes = BitVecVal(int(bitvec.as_long()), bitvec.size())
    # Perform SHA-256 hashing
    hash_bytes = sha256(bitvec_bytes.as_long().to_bytes(bitvec.size() // 8, byteorder='big'))
    # Convert hash bytes back to BitVec
    return BitVecVal(int.from_bytes(hash_bytes, byteorder='big'), 256)

# Run the input through SHA-256 to get a symbolic BitVec hash digest
digest = symbolic_sha256(input_message)

# Run through SHA-256 a second time
second_digest = symbolic_sha256(digest)

# Create a variable for the hash output, which is unknown
hash_guess = BitVec('hash_guess', 256)

# Add solver constraint that this variable must equal the digest
solver.add(hash_guess == second_digest)

# Add a constraint to the solver that the beginning of the hash must have some number of zeros, e.g., 64
hash_beginning = Extract(63, 0, hash_guess)
solver.add(hash_beginning == 0)

# Check if the problem has a solution, and print it
if solver.check() == sat:
    model = solver.model()
    print("Solution found:")
    print(model)
    nonce_value = model[nonce].as_long()
    print(f"Nonce: {nonce_value}")
else:
    print("No solution found.")

ModuleNotFoundError: No module named 'z3'